# Performance Evaluation

## Helper Functions

In [3]:
### install necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [2]:
### defining labels
class_labels = ["unverifiable", "false", "mostly false", "half true", "mostly true", "true"]
class_2_index = {label: i for i, label in enumerate(class_labels)}
index_2_class = {i: label for i, label in enumerate(class_labels)}
num_labels = [class_2_index[label] for label in class_labels] # [0, 1, 2, 3, 4, 5]

In [235]:
### functions to generate confusion matrix and metrics
def generate_cm(y_true, y_pred):
    '''
    Calculate the confusion matrix with the cost matrix
    '''
    cm = confusion_matrix(y_true, y_pred)
    return cm

def generate_metrics(y_true, y_pred):
    '''
    Calculate the weighted F1 score
    '''
    weighted_precision = precision_score(y_true, y_pred, average='weighted', sample_weight=None, labels=num_labels)
    weighted_recall = recall_score(y_true, y_pred, average='weighted', sample_weight=None, labels=num_labels)
    weighted_f1 = f1_score(y_true, y_pred, average='weighted', sample_weight=None, labels=num_labels)
    metrics = pd.DataFrame({'Precision': weighted_precision, 'Recall': weighted_recall, 'F1': weighted_f1}, index=class_labels)
    return metrics

## Label Generation

In [2]:
### load the data
df = pd.read_csv('../data/[FINAL] Pilot - Pilot Claims copy.csv')
statements = df['statement'].to_list()
statement_dates = df['statement_date'].to_list()
statement_originators = df['statement_originator'].to_list()
gold = df['verdict'].to_list()

'''
Creates a list of strings that contain the statement, source, and date in the following format:
Statement: {statement}
According to: {source}
Date: {date}
'''
statements_agg = [f"""Statement: {statement} \n According to: {source} \n Date: {date}""" for statement, source, date in zip(statements, statement_originators, statement_dates)]

In [38]:
### generate results
def generate_results(statements: list, 
                     old_file: str = None, 
                     new_file: str = None):
    '''
    generate and store results for a list of statements
    make sure to:
    (1) load the old results if they exist
    (2) initialize the pipeline and define LM before running
    (3) run %%capture to suppress output

    '''
    # load results if it exists
    results = []
    if os.path.exists(old_file):
        with open(old_file, 'rb') as f:
            results = pickle.load(f)

    for index, statement in enumerate(tqdm(statements)):
        if len(results) <= index+1 and type(results[index]) != int:
            continue
        verdict = None

        # retry 5 times if there is an error
        for i in range(5):
            try:
                verdict, confidence, reasoning, claims = pipeline.fact_check(statement)
            except Exception as e:
                print(f"Error {e}: retrying for statement {index}, attempt {i+1}")
                continue 
            break   
        
        if verdict is None:
            results.append(index)
        else:
            results.append((verdict, confidence, reasoning, claims))
        with open(new_file, 'wb') as f:
            pickle.dump(results, f)

In [39]:
### generate results dataframe
def generate_results_df(results_filename: str,
                        df_filename: str):
    '''
    convert results to a dataframe and save it
    '''
    if os.path.exists(results_filename):
        with open(results_filename, 'rb') as f:
            results = pickle.load(f)
    results_df = pd.DataFrame(data=results, columns=['verdict', 'confidence', 'reasoning', 'claims'])
    results_df.to_csv(df_filename)
    return results_df

### Gemini

In [1]:
%reload_ext autoreload
%autoreload 2

### load fact-checking pipeline
import dotenv
import sys
import dspy
import os
sys.path.append('../pipeline_v2/')
import main 
dotenv.load_dotenv('../.env')

# initialize search provider
main.NUM_SEARCH_RESULTS = 10 # Number of search results to retrieve
main.SCRAPE_TIMEOUT = 5 # Timeout for scraping a webpage (in seconds)
search_provider = main.SearchProvider(provider="duckduckgo")

# initialize DSPy
lm = dspy.LM('gemini/gemini-1.5-flash', api_key=os.getenv('GOOGLE_GEMINI_API_KEY'), cache=False)
#lm = dspy.LM('ollama_chat/mistral', api_base='http://localhost:11434', api_key='')
dspy.settings.configure(lm=lm)

# initialize pipeline
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
main.VERBOSE = True # Print intermediate results
main.INTERACTIVE = False # Allow the user to provide feedback
main.USE_BM25 = True # Use BM25 for retrieval (in addition to cosine similarity)
main.BM25_WEIGHT = 0.5 # Weight for BM25 in the hybrid retrieval

pipeline = main.FactCheckPipeline(
    search_provider=search_provider,
    model_name=lm,
    embedding_model=embedding_model,
    retriever_k=2
)

/opt/homebrew/Caskroom/miniconda/base/envs/ACL/lib/python3.10/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [299]:
print(main.VERDICTS)

['supported', 'unsupported']


### Mistral

In [35]:
%reload_ext autoreload
%autoreload 2
import dotenv
import sys
import dspy
import os
sys.path.append('../pipeline_v2/')
import main 
dotenv.load_dotenv('../.env')

# Initialize search provider
main.NUM_SEARCH_RESULTS = 10 # Number of search results to retrieve
main.SCRAPE_TIMEOUT = 5 # Timeout for scraping a webpage (in seconds)
search_provider = main.SearchProvider(provider="duckduckgo")

# Initialize DSPy
lm = dspy.LM('ollama_chat/mistral', api_base='http://localhost:11434', api_key='')
dspy.settings.configure(lm=lm)

# Initialize pipeline
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
main.VERBOSE = False # Print intermediate results
main.INTERACTIVE = False # Allow the user to provide feedback
main.USE_BM25 = True # Use BM25 for retrieval (in addition to cosine similarity)
main.BM25_WEIGHT = 0.5 # Weight for BM25 in the hybrid retrieval

pipeline = main.FactCheckPipeline(
    search_provider=search_provider,
    model_name=lm,
    embedding_model=embedding_model,
    retriever_k=2
)


## AVeriTeC

In [24]:
import json

# Open and read the JSON file
with open('data_dev.json', 'r') as file:
    averitec = json.load(file)

averitec_20 = averitec[:20]

In [68]:
averitec_20

[{'claim': 'In a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.',
  'required_reannotation': False,
  'label': 'Refuted',
  'justification': 'The answer and sources show that the claim was published in a fake news site so the claim is refuted.',
  'claim_date': '31-10-2020',
  'speaker': None,
  'original_claim_url': None,
  'fact_checking_article': 'https://web.archive.org/web/20201130144023/https://checkyourfact.com/2020/11/03/fact-check-sean-connery-letter-steve-jobs-apple-1998/',
  'reporting_source': 'Facebook',
  'location_ISO_code': None,
  'claim_types': ['Event/Property Claim'],
  'fact_checking_strategies': ['Written Evidence'],
  'questions': [{'question': 'Where was the claim first published',
    'answers': [{'answer': 'It was first published on Sccopertino',
      'answer_type': 'Abstractive',
      'source_url': 'https://web.archive.org/web/20201129141238/https://scoopertino.com/exposed-the-imac-disaster-that-almost-was/',
      'source_medi

In [71]:
with open("data_dev_1.json", "w") as final:
	json.dump(averitec_20, final)

In [85]:
averitec_results = []
for s in statements:
    verdict, confidence, reasoning, claims = pipeline.fact_check(s)
    questions = [claim.components[j].question for claim in claims for j in range(len(claim.components))] # List of questions
    answers = [claim.components[j].answer.text for claim in claims for j in range(len(claim.components))] # List of corresponding answers
    urls = ['x' for i in range(len(questions))] # List of corresponding URLs
    pred_label = verdict
    print('='*20)
    print(pred_label)
    averitec_results.append(format_prediction(questions, answers, urls, pred_label))

20

In [84]:
[c['pred_label'] for c in averitec_results]

['Refuted',
 'Refuted',
 'Refuted',
 'Refuted',
 'Refuted',
 'Not Enough Evidence',
 'Supported',
 'Supported',
 'Not Enough Evidence',
 'Supported',
 'Conflicting Evidence/Cherry-picking',
 'Not Enough Evidence',
 'Refuted',
 'Not Enough Evidence',
 'Not Enough Evidence',
 'Not Enough Evidence',
 'Not Enough Evidence',
 'Not Enough Evidence',
 'Not Enough Evidence',
 'Refuted']

In [207]:
df = pd.read_csv('../data/pilot_updated.csv')

In [211]:
df.verdict.value_counts()

verdict
FALSE            40
MOSTLY FALSE     31
MOSTLY TRUE      27
TRUE             25
HALF TRUE        17
PANTS ON FIRE     9
UNVERIFIABLE      1
Name: count, dtype: int64

In [86]:
with open("my_data.json", "w") as final:
	json.dump(averitec_results, final)

# PolitiFact Scraper

In [194]:
import requests
import re
from bs4 import BeautifulSoup
import dateutil.parser as dparser
def scrape_politifact(url):
    """Scrapes text and metadata from a given website URL."""
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            article_element = soup.find('article', class_='m-textblock')
            statement = soup.find('div', class_='m-statement__quote').get_text()
            statement = re.sub(r'\s+', ' ', statement).strip()

            verdict = soup.find('div', class_='m-statement__meter').find('img').get('alt').upper().replace('-',' ')
            if verdict == 'BARELY TRUE':
                verdict = 'MOSTLY FALSE'
            author = soup.find('div',class_="m-statement__author")
            statement_originator = author.find('a',class_='m-statement__name').get_text()
            statement_originator = re.sub(r'\s+', ' ', statement_originator).strip()
            
            statement_date = author.find('div',class_='m-statement__desc').get_text()
            statement_date = re.sub(r'\s+', ' ', statement_date).strip()
            statement_date = dparser.parse(statement_date,fuzzy=True).strftime('%m/%d/%Y')
            if article_element:
                section_element = article_element.find('section', class_='o-pick')
                if section_element:
                    section_element.decompose()
                article_text = article_element.get_text()
                article_text = article_text.replace(u'\xa0', u' ')
                reasoning = re.sub(r'\s+', ' ', article_text).strip()
            return verdict, statement_originator, statement, statement_date, reasoning, url
    except Exception as e:
        print(f"Error during website scraping: {e}")
        return None

In [195]:
urls = ['https://www.politifact.com/factchecks/2025/jan/30/karoline-leavitt/no-the-us-did-not-spend-50-million-to-fund-condoms/',
        'https://www.politifact.com/factchecks/2025/jan/30/elizabeth-warren/whats-the-evidence-rfk-jr-profited-off-vaccine-law/',
        'https://www.politifact.com/factchecks/2024/nov/04/winsenate/democratic-ad-misleads-on-kari-lakes-support-for-s/',
        ]

In [196]:
factcheck_urls = [
    "https://www.politifact.com/factchecks/list/?ruling=half-true",
    "https://www.politifact.com/factchecks/list/?ruling=true",
    "https://www.politifact.com/factchecks/list/?ruling=false",
    "https://www.politifact.com/factchecks/list/?ruling=mostly-true",
    "https://www.politifact.com/factchecks/list/?ruling=barely-true",
    "https://www.politifact.com/factchecks/list/?ruling=pants-fire",
]


In [204]:
new_data = pd.DataFrame(data, columns=['verdict', 'statement_originator', 'statement', 'statement_date', 'reasoning', 'url'])

In [203]:
data = []
for ruling in factcheck_urls:
    response = requests.get(ruling, timeout=5)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        article_element = soup.find_all('li', class_='o-listicle__item')
        print(len(article_element))
    for i in range(30):
        ref = article_element[i].find('div', class_='m-statement__quote').find('a').get('href')
        url = f'https://www.politifact.com{ref}'
        print(url)
        news = scrape_politifact(url)
        data.append(news)

30
https://www.politifact.com/factchecks/2025/jan/30/karoline-leavitt/karoline-leavitt-blames-biden-for-egg-shortage-but/
https://www.politifact.com/factchecks/2025/jan/30/elizabeth-warren/whats-the-evidence-rfk-jr-profited-off-vaccine-law/
https://www.politifact.com/factchecks/2025/jan/29/roy-cooper/roy-cooper-says-hurricane-helene-relief-bill-took/
https://www.politifact.com/factchecks/2025/jan/17/dana-loesch/are-north-carolinas-helene-victims-still-required/
https://www.politifact.com/factchecks/2025/jan/06/social-media/poll-shows-trump-won-18-24-year-olds-in-wisconsin/
https://www.politifact.com/factchecks/2024/dec/10/tweets/how-much-temporary-housing-has-fema-provided-to-nc/
https://www.politifact.com/factchecks/2024/nov/04/winsenate/democratic-ad-misleads-on-kari-lakes-support-for-s/
https://www.politifact.com/factchecks/2024/nov/01/make-the-road-nevada/nevada-political-action-committee-gets-some-right/
https://www.politifact.com/factchecks/2024/oct/30/viral-image/an-abc-news-aff

# LLM-AggreFact 

In [ ]:
import pandas as pd
from datasets import load_dataset
from sklearn.metrics import balanced_accuracy_score
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### Pipeline

In [16]:
from termcolor import colored
from utils import print_header

def print_final_result(statement, verdict, confidence, reasoning, gold_verdict=None):
    print("\nFinal Fact-Check Result:")
    print_header(f"Statement: {colored(statement, 'white')}", level=1)
    print_header(f"Overall Verdict: {colored(verdict, 'green')}", level=1)
    print_header(f"Overall Confidence: {colored(str(confidence), 'yellow')}", level=1)
    print_header(f"Overall Reasoning: {colored(reasoning, 'cyan')}", level=1)
    if gold_verdict: print_header(f"Gold Verdict: {colored(gold_verdict, 'green')}", level=1)

### Load data
# output_file = args.output_file
# if os.path.exists(output_file):
#     df = pd.read_pickle(output_file)
# Set custom constants for whole pipeline
# main.VERBOSE = False # Print intermediate results
main.VERDICTS=["unsupported", "supported"]

model = 'gemini'
num_trials = 3
lm = dspy.LM('gemini/gemini-1.5-flash', api_key=os.getenv('GOOGLE_GEMINI_API_KEY'), cache=False)
output_file = f'aggrefact_{model}.pkl'
if os.path.exists(output_file):
    df = pd.read_pickle(output_file)
else: 
    df = pd.DataFrame(load_dataset("lytang/LLM-AggreFact")['test'])
    # Drop unneeded columns
    df = df[df['dataset'].isin(['RAGTruth','ExpertQA','Lfqa','ClaimVerify'])]
    df = df.groupby('dataset').apply(lambda x: x.sample(frac=0.01, random_state=42))
    df.reset_index(drop=True, inplace=True)
dspy.settings.configure(lm=lm, temperature=0.3)

# If column doesn't exist, create it
if f'{model}_results' not in df.columns: df[f'{model}_results'] = None
df[f'{model}_results'] = df[f'{model}_results'].astype(object)
display(df.head())

for index in tqdm(range(len(df))):
    # If results already exist, skip if num_trials is reached
    if df.iloc[index][f'{model}_results'] is not None: 
        if len(df.loc[index, f'{model}_results']) == num_trials:
            print(f"Skipping row {index} because {num_trials}/{num_trials} trials completed")
            continue
        else:
            print(f"Running row {index} because {len(df.loc[index, f'{model}_results'])}/{num_trials} trials completed")
            results = df.loc[index, f'{model}_results']
    else: 
        print(f"Running row {index} because 0/{num_trials} trials completed")
        results = []

    for trial_i in tqdm(range(num_trials-len(results)), leave=False):
        source_doc = df.iloc[index]['doc']
        pipeline = main.FactCheckPipeline(
            search_provider=main.SearchProvider(provider="duckduckgo"),
            model_name=lm,
            embedding_model=main.EMBEDDING_MODEL,
            context=[main.Document(content=source_doc, metadata={"source": "source"})],
            retriever_k=5
        )

        statement = df.iloc[index]['claim']
        gold_verdict = df.iloc[index]['label']
        try: 
            verdict, confidence, reasoning, claims = pipeline.fact_check(
                # statement=f"According to {statement_originator} on {statement_date}, {statement}", 
                statement=statement, 
                web_search=False
                # statement=statement, 
                # context=f"Statement Originator: {statement_originator}, Date Claim Was Made: {statement_date}"
            )
        except Exception as e: 
            if 'Failed to extract claim' in str(e): 
                continue

        print_final_result(statement, verdict, confidence, reasoning, gold_verdict)
        results.append({
            'verdict': verdict,
            'confidence': confidence,
            'reasoning': reasoning,
            'claims': claims
        })
        df.at[index, f'{model}_results'] = results
        df.to_pickle(output_file)

,dataset,doc,claim,label,contamination_identifier,gemini_results
0,ClaimVerify,Using acrylic paint is a fun and economical wa...,You can use a single acrylic paint color or mi...,1,LLM-AggreFact:PuKGySjD1LKX354etBOla3Ke9VjVBmYR...,"[{'verdict': 'supported', 'confidence': 1.0, '..."
1,ClaimVerify,"Recipe by rjholtz\nUpdated on April 23, 2023\n...",You can also grill tilapia in foil or use othe...,1,LLM-AggreFact:nGjDY4zLhqIU5KzXn5YqfKdQDL72ru6u...,"[{'verdict': 'supported', 'confidence': 0.95, ..."
2,ClaimVerify,Listen to Chuck Swindoll’s overview of First J...,The author of the book of First John is not id...,1,LLM-AggreFact:3EjFNtDyEESiwE3PKH4eILjTpzVzoQRv...,"[{'verdict': 'supported', 'confidence': 0.9, '..."
3,ClaimVerify,Agender is defined as not having a gender. Som...,Agender people may use gender-neutral pronouns...,0,LLM-AggreFact:kgftCc3SYJipb7CnIJHAuTjCgljJ4LhQ...,"[{'verdict': 'unsupported', 'confidence': 0.6,..."
4,ClaimVerify,"2000, Comedy/Drama, 1h 46m\nCritics Consensus\...","The cast of O Brother, Where Art Thou? include...",0,LLM-AggreFact:31kK2h5aC3tGW4bFrV3z5yXj7vznzrlV...,"[{'verdict': 'partially supported', 'confidenc..."


  0%|          | 0/231 [00:00<?, ?it/s]

Skipping row 0 because 3/3 trials completed
Skipping row 1 because 3/3 trials completed
Skipping row 2 because 3/3 trials completed
Skipping row 3 because 3/3 trials completed
Skipping row 4 because 3/3 trials completed
Skipping row 5 because 3/3 trials completed
Skipping row 6 because 3/3 trials completed
Skipping row 7 because 3/3 trials completed
Skipping row 8 because 3/3 trials completed
Skipping row 9 because 3/3 trials completed
Skipping row 10 because 3/3 trials completed
Skipping row 11 because 3/3 trials completed
Skipping row 12 because 3/3 trials completed
Skipping row 13 because 3/3 trials completed
Skipping row 14 because 3/3 trials completed
Skipping row 15 because 3/3 trials completed
Skipping row 16 because 3/3 trials completed
Skipping row 17 because 3/3 trials completed
Skipping row 18 because 3/3 trials completed
Skipping row 19 because 3/3 trials completed
Skipping row 20 because 3/3 trials completed
Skipping row 21 because 3/3 trials completed
Skipping row 22 beca

  0%|          | 0/2 [00:00<?, ?it/s]

===== Starting Fact Check Pipeline =====
 Original Statement: Passage 3 provides information on the varying pay structures among different companies, with some offering hourly and commission pay rates, while others may use a combination of both. 
  ===== Atomic Claim Extraction =====
   Extracted Claims (1):  
     1. Passage 3 indicates that different companies utilize varying pay structures, including hourly rates, commission-based rates, or a combination of both. 
    ===== Question Generation [1/1] =====
       Decomposed Components (Questions + Search Queries) (1): 
         1. Question: What types of pay structures are mentioned in Passage 3 as being used by different companies? 
            Search Queries: ['Passage 3 pay structures companies'] 
      ===== Question Answering for Component [1/1] =====
         Question: What types of pay structures are mentioned in Passage 3 as being used by different companies? 
         Search Queries: ['Passage 3 pay structures companies'] 
 

...

===== Starting Fact Check Pipeline =====
 Original Statement: Cover the grill and cook for 5 minutes. 
  ===== Atomic Claim Extraction =====
   Extracted Claims (1):  
     1. Cover the grill and cook for 5 minutes. 
    ===== Question Generation [1/1] =====
       Decomposed Components (Questions + Search Queries) (0): 
    ===== Claim Evaluation =====
       Claim: Cover the grill and cook for 5 minutes. 
       Verdict: unsupported 
       Confidence: 0.0 
       Reasoning: There are no question-answer pairs provided to evaluate the claim "Cover the grill and cook for 5 minutes".  Therefore, it's impossible to determine if the claim is supported or unsupported by evidence. 
       Evidence Analysis 
===== Breakdown of Claims and Components =====
   Claim 1 
     Text: Cover the grill and cook for 5 minutes. 
     Verdict: unsupported 
     Confidence: 0.0 
     Reasoning: There are no question-answer pairs provided to evaluate the claim "Cover the grill and cook for 5 minutes".  The

  0%|          | 0/3 [00:00<?, ?it/s]

===== Starting Fact Check Pipeline =====
 Original Statement: They lauded the customer service and the ability to customize their order. 
  ===== Atomic Claim Extraction =====
   Extracted Claims (2):  
     1. Customers lauded the customer service. 
     2. Customers lauded the ability to customize their order. 
    ===== Question Generation [1/2] =====
       Decomposed Components (Questions + Search Queries) (1): 
         1. Question: What specific aspects of customer service did customers praise in reviews or feedback for this company/product? 
            Search Queries: ['customer reviews [company name/product name] customer service', '[company name/product name] customer service feedback analysis'] 
      ===== Question Answering for Component [1/1] =====
         Question: What specific aspects of customer service did customers praise in reviews or feedback for this company/product? 
         Search Queries: ['customer reviews [company name/product name] customer service', '[c

  0%|          | 0/3 [00:00<?, ?it/s]

===== Starting Fact Check Pipeline =====
 Original Statement: The cause remains unknown and the ship has increased cleaning and disinfection procedures. 
  ===== Atomic Claim Extraction =====
   Extracted Claims (2):  
     1. The cause remains unknown. 
     2. The ship has increased cleaning and disinfection procedures. 
    ===== Question Generation [1/2] =====
       Decomposed Components (Questions + Search Queries) (2): 
         1. Question: What investigations have been conducted to determine the cause of the issue? 
            Search Queries: ['investigation report', 'cause of incident report'] 
         2. Question: What specific challenges or limitations hindered the identification of the cause? 
            Search Queries: ['investigation limitations', 'unidentified cause explanation'] 
      ===== Question Answering for Component [1/2] =====
         Question: What investigations have been conducted to determine the cause of the issue? 
         Search Queries: ['investig

### Baseline

In [79]:
# Initialize baseline
VERDICTS = ['supported','unsupported']
class StatementFactCheckerSignature(dspy.Signature):
    f"""Fact check the given statement and context into one of the following verdicts: {", ".join(VERDICTS)}"""
    statement = dspy.InputField(desc="Statement to evaluate")
    verdict = dspy.OutputField(desc=f"Truthful classification of the statement into one of the following verdicts: {', '.join(main.VERDICTS)}")

class StatementFactChecker(dspy.Module):
    def __init__(self):
        super().__init__()
        self.fact_check = dspy.ChainOfThought(StatementFactCheckerSignature)

    def forward(self, statement: str):
        result = self.fact_check(statement=statement)
        verdict = result["verdict"]
        reasoning = result["reasoning"]
        return verdict, reasoning
    
baseline = StatementFactChecker()

if os.path.exists(output_file):
    df_1 = pd.read_pickle(output_file)

# If baseline results column doesn't exist, create it
if f'{model}_baseline_results' not in df_1.columns: df_1[f'{model}_baseline_results'] = None
df_1[f'{model}_baseline_results'] = df_1[f'{model}_baseline_results'].astype(object)

for index in tqdm(range(len(df_1))):
    # If results already exist, skip if num_trials is reached
    if df_1.loc[index, f'{model}_baseline_results'] is not None: 
        if len(df_1.loc[index, f'{model}_baseline_results']) == num_trials:
            print(f"Skipping row {index} because {num_trials}/{num_trials} trials completed")
            continue
        else:
            print(f"Running row {index} because {len(df_1.loc[index, f'{model}_baseline_results'])}/{num_trials} trials completed")
            results = df_1.loc[index, f'{model}_baseline_results']
    else: 
        print(f"Running row {index} because 0/{num_trials} trials completed")
        results = []

    for trial_i in tqdm(range(num_trials-len(results)), leave=False):
        statement = df_1.iloc[index]['claim']
        gold_verdict = df_1.iloc[index]['label']

        verdict, reasoning = baseline(f"{statement}")
        print_final_result(statement, verdict, 'N/A', reasoning, gold_verdict)

        results.append({
            'verdict': verdict,
            'reasoning': reasoning
        })

        df_1.at[index, f'{model}_baseline_results'] = results
        df_1.to_pickle(output_file)

  0%|          | 0/231 [00:00<?, ?it/s]

Skipping row 0 because 3/3 trials completed
Skipping row 1 because 3/3 trials completed
Skipping row 2 because 3/3 trials completed
Skipping row 3 because 3/3 trials completed
Skipping row 4 because 3/3 trials completed
Skipping row 5 because 3/3 trials completed
Skipping row 6 because 3/3 trials completed
Skipping row 7 because 3/3 trials completed
Skipping row 8 because 3/3 trials completed
Skipping row 9 because 3/3 trials completed
Skipping row 10 because 3/3 trials completed
Skipping row 11 because 3/3 trials completed
Skipping row 12 because 3/3 trials completed
Skipping row 13 because 3/3 trials completed
Skipping row 14 because 3/3 trials completed
Skipping row 15 because 3/3 trials completed
Skipping row 16 because 3/3 trials completed
Skipping row 17 because 3/3 trials completed
Skipping row 18 because 3/3 trials completed
Skipping row 19 because 3/3 trials completed
Skipping row 20 because 3/3 trials completed
Skipping row 21 because 3/3 trials completed
Skipping row 22 beca

  0%|          | 0/2 [00:00<?, ?it/s]


Final Fact-Check Result:
   Statement: In a bowl, mix together seasoned salt, black pepper, garlic powder, onion powder, paprika, Worcestershire sauce, and smoke flavoring until thoroughly combined. 
   Overall Verdict: supported 
   Overall Confidence: N/A 
   Overall Reasoning: The statement describes a straightforward process of combining several ingredients in a bowl.  There's no inherent falsehood or unsupported claim within the instructions.  The process is entirely feasible and commonly done in cooking. 
   Gold Verdict: 1 

Final Fact-Check Result:
   Statement: In a bowl, mix together seasoned salt, black pepper, garlic powder, onion powder, paprika, Worcestershire sauce, and smoke flavoring until thoroughly combined. 
   Overall Verdict: supported 
   Overall Confidence: N/A 
   Overall Reasoning: The statement describes a straightforward process of combining several ingredients in a bowl.  There's no inherent falsehood or unsupported claim within the instructions.  The proc

  0%|          | 0/3 [00:00<?, ?it/s]

...


Final Fact-Check Result:
   Statement: Cover the grill and cook for 5 minutes. 
   Overall Verdict: unsupported 
   Overall Confidence: N/A 
   Overall Reasoning: The statement "Cover the grill and cook for 5 minutes" is a directive or instruction, not a factual statement that can be evaluated as true or false.  Therefore, it cannot be classified as supported or unsupported. 
   Gold Verdict: 1 

Final Fact-Check Result:
   Statement: Cover the grill and cook for 5 minutes. 
   Overall Verdict: unsupported 
   Overall Confidence: N/A 
   Overall Reasoning: The statement "Cover the grill and cook for 5 minutes" is a directive or instruction, not a factual statement that can be evaluated as true or false.  Therefore, it cannot be classified as supported or unsupported. 
   Gold Verdict: 1 

Final Fact-Check Result:
   Statement: Cover the grill and cook for 5 minutes. 
   Overall Verdict: unsupported 
   Overall Confidence: N/A 
   Overall Reasoning: The statement "Cover the grill and c

  0%|          | 0/3 [00:00<?, ?it/s]


Final Fact-Check Result:
   Statement: They lauded the customer service and the ability to customize their order. 
   Overall Verdict: supported 
   Overall Confidence: N/A 
   Overall Reasoning: The statement presents a positive assessment of a service, highlighting aspects like customer service and customization options.  Without further context or evidence to contradict this, the statement can be considered supported. 
   Gold Verdict: 1 

Final Fact-Check Result:
   Statement: They lauded the customer service and the ability to customize their order. 
   Overall Verdict: supported 
   Overall Confidence: N/A 
   Overall Reasoning: The statement presents a positive assessment of a service, highlighting aspects like customer service and customization options.  Without further context or evidence to the contrary, the statement can be considered supported. 
   Gold Verdict: 1 

Final Fact-Check Result:
   Statement: They lauded the customer service and the ability to customize their o

  0%|          | 0/3 [00:00<?, ?it/s]


Final Fact-Check Result:
   Statement: The cause remains unknown and the ship has increased cleaning and disinfection procedures. 
   Overall Verdict: unsupported 
   Overall Confidence: N/A 
   Overall Reasoning: The statement presents two separate pieces of information: the cause of something is unknown, and a ship has increased its cleaning and disinfection procedures.  While these are related in that increased cleaning might be a response to an unknown cause, the statement itself doesn't offer evidence to support or refute either claim independently.  We cannot verify the existence of an unknown cause, nor can we independently verify the increased cleaning procedures. 
   Gold Verdict: 1 

Final Fact-Check Result:
   Statement: The cause remains unknown and the ship has increased cleaning and disinfection procedures. 
   Overall Verdict: unsupported 
   Overall Confidence: N/A 
   Overall Reasoning: The statement presents two separate pieces of information: the cause of something 